In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import pandas as pd
import os
import requests
import re

### The Punch Newspaper

In [24]:
def punch_news():
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')

    c = DesiredCapabilities.CHROME
    c["pageLoadStrategy"] = "none"
    #set chromodriver.exe path
    driver = webdriver.Chrome(r'C:\Users\HP\Downloads\news\News_station_analysis\chromedriver.exe',desired_capabilities=c)
    #explicit wait
    w = WebDriverWait(driver, 20)
    #launch URL
    driver.get("https://punchng.com/topics/news/")
    driver.implicitly_wait(20)
    time.sleep(3)
    #driver.implicitly_wait(20)
    #expected condition
    w.until(EC.presence_of_element_located((By.CLASS_NAME, 'post-title')))
    #JavaScript Executor to stop page load

    driver.execute_script("window.stop();")
    print('First huddle')

    content = []
    contents =driver.find_elements_by_class_name("post-title")
    for con in contents:
        cont = con.get_attribute('innerHTML')
        content.append(cont)
    af = pd.DataFrame(content,columns =['content'])
    af.content = af.content.apply(lambda x: x.replace('<a href=', ''))
    af.content = af.content.apply(lambda x: x.replace('</a>', ''))
    af.content = af.content.apply(lambda x: x.replace('>', '|'))
    af = af['content'].str.split("|",n = 3, expand = True)
    af.columns = ['link','title']
    af = af.drop_duplicates(subset=["link"], keep='first')
    print('Second huddle')
    full_contents = []
    dates = []
    by = []
    def all_news(ev):
        h = WebDriverWait(driver, 20)
        full = []
        timed = []
        print('Pages extraction in progress')

        driver.get(ev)
        time.sleep(4)
        driver.implicitly_wait(20)
        h.until(EC.presence_of_element_located((By.CLASS_NAME, 'post-content')))
        #JavaScript Executor to stop page load
        driver.execute_script("window.stop();")
        full_content = driver.find_elements_by_class_name("post-content")
        for conten in full_content:
            co = conten.get_attribute('innerText')
            co1 = co.replace('\n\n',' ')
            co2 = co1.replace('\n',' ')
            co3 = co2.split(',', 1)
            full.append(co3)

        date = driver.find_elements_by_class_name("col-lg-4")
        for dat in date:
            dat1= dat.get_attribute('innerText')
            dat2 = dat1.replace('By\xa0\n','')
            timed.append(dat2)
        full_contents.append(full[0])
        dates.append(timed[0])
        by.append(timed[1])

    m =af.link.to_list()
    m =  [item.replace('"', '') for item in m]
    for o in m:
        all_news(o)

    driver.quit()

    aa = pd.DataFrame({'Title':af.title,'Full_content': full_contents,'Date':dates,'Author':by,'Source_link':af.link})
    ff  =aa['Full_content'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))#unlist the full_content column

    aa['Words_count'] = ff.str.split().str.len()#counts the full_content
    n = open("negative-words.txt", "r")
    p = open("positive-words.txt", "r")
    n_word = n.read()
    p_word = p.read()
    n.close()
    p.close
    n_word=n_word.replace('\n',',')
    n_word = re.sub("[^\w]", " ", n_word).split()
    p_word=p_word.replace('\n',',')
    p_word = re.sub("[^\w]", " ", p_word).split()
    aa['Full_content'] = ff
    #df['word_overlap'] = [set(x[0].split()) & set(x[1].split()) for x in df.values]
    def negative_words(x):
        negative_score = 0
        for word in n_word:
            if word in x:
                negative_score += 1
        return negative_score

    def positive_words(x):
        positive_score = 0
        for word in p_word:
            if word in x:
                positive_score += 1
        return positive_score
    aa['Negative_words'] = aa['Full_content'].apply(lambda x : negative_words(x))
    aa['Positive_words'] = aa['Full_content'].apply(lambda x : positive_words(x))
    aa['Sentence_count'] = aa['Full_content'].str.count('[\w][\.!\?]')
    aa['Sentiment'] = round((aa['Positive_words'] - aa['Negative_words']) / aa['Words_count'], 2)
    aa['News_type'] = ['Bad News' if x < 0 else 'Good News' if x > 0 else 'Neutral' for x in aa.Sentiment]
    return aa

In [ ]:
punch_news()

In [31]:
!git push origin master


error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/George-Michael-Dagogo/News_station_analysis'
